In [1]:
import json
import random
from random import choice
from tkinter import _flatten
from tqdm import tqdm

In [2]:
allentity = []

In [3]:
# get all entity
def getcandidate(filepath):
    entity = []
    with open(filepath) as f:
        diclist = json.load(f)['entries']
        for dic in diclist:
            dic = list(dic.values())[0]
            for ops in dic["modifiedtripleset"]:
                entity.append(' '.join(ops['subject'].split('_')).replace('"', '').lower())
                entity.append(' '.join(ops['object'].split('_')).replace('"', '').lower())
    return entity

In [4]:
# change entity
def change(wd):
    if random.random() < 1:
        return choice(allentity), True
    else:
        return wd, False

In [5]:
# change sentence and mark where changed
def makenoisy(gold, datadiclist, datastr):
    noisytext = [gold]
    # sum = 0
    entitys = []
    # get all entity in sentence 
    for datadic in datadiclist:
        entitys.append(' '.join(datadic['subject'].lower().split('_')))
        entitys.append(' '.join(datadic['object'].lower().split('_')))
    # split by entity
    for entity in entitys:
        noisytext = [x.split(' '+entity+' ') for x in noisytext]
        z = []
        for x in noisytext:
            tmp = [x[0].strip()]
            for y in x[1:]:
                tmp.append(entity)
                tmp.append(y.strip())
            z.append(tmp)
        noisytext = [x for x in list(_flatten(z)) if x]

    changedid = []
    originid = []
    retext = []

    # mark where changed
    # wd is not a word but words
    for i, wd in enumerate(noisytext):
        if wd in entitys:
            nwd, wh = change(wd)
            if wh:
                changedid.append(i)
                originid.append(entitys.index(wd))
                wd = nwd
        retext.append(wd)
    
    prelen = 0
    errortextpos = ''
    corredatapos = ''

    # from index to position
    for i, wd in enumerate(retext):
        if i in changedid:
            errortextpos+='\t'+str(prelen)+' '+str(prelen+len(wd.split(' ')))
        prelen += len(wd.split(' '))
    # data position
    for i in originid:
        lis = datastr.split('> '+entitys[i])
        corredatapos+='\t'+str(len(lis[0].strip().split(' ')))+' ' + str(len(lis[0].strip().split(' '))+len(entitys[i].split(' ')))
    return ' '.join(retext), errortextpos[1:], corredatapos[1:]

In [6]:
# change file, just some form
def makepseudo(filepath):
    with open(filepath) as f:
        fsrcdata = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.src.data', 'w')
        fsrctext = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.src.text', 'w')
        ftgttext = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.text', 'w')
        ftgtdatapos = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.datapos', 'w')
        ftgttextpos = open('../pseudo/' + filepath.split('_')[-1].split('.')[0] + '.tgt.textpos', 'w')

        diclist = json.load(f)['entries']
        for dic in tqdm(diclist):
            dic = list(dic.values())[0]
            dic.pop('originaltriplesets')
            textlist = dic.pop('lexicalisations')
            textlist = [x['lex'] for x in textlist]
            datastr = '<category> '+dic['category']+' <type> '+dic['shape_type']+' <sep>'
            for datadic in dic["modifiedtripleset"]:
                datastr += ' <s> '+' '.join(datadic['subject'].split('_')).lower()+' <p> '+datadic['property']+' <o> '+' '.join(datadic['object'].split('_')).lower()
            
            datastr = datastr + ' ' + '<end>'
            for text in textlist:
                text = text.lower()
                ftgttext.write(text+'\n')
                noisytext, errortextpos, corredatapos = makenoisy(text, dic["modifiedtripleset"], datastr)
                fsrctext.write(noisytext + '\n')
                fsrcdata.write(datastr + '\n')
                ftgtdatapos.write(corredatapos + '\n')
                ftgttextpos.write(errortextpos + '\n')
        ftgttext.close() 
        fsrcdata.close()
        fsrctext.close()
        ftgttextpos.close()
        ftgtdatapos.close()

In [7]:
allentity=getcandidate('webnlg_release_v2.1_dev.json') + getcandidate('webnlg_release_v2.1_test.json') + getcandidate('webnlg_release_v2.1_train.json')


In [8]:
makepseudo('webnlg_release_v2.1_test.json')
makepseudo('webnlg_release_v2.1_dev.json')
makepseudo('webnlg_release_v2.1_train.json')

100%|██████████| 12876/12876 [00:01<00:00, 8824.58it/s]


In [9]:
len(allentity)

93588

# to classify

In [10]:
sss = ['train', 'dev', 'test']
for ss in sss:
    with open('../pseudo/' + ss + '.tgt.textpos') as f:
        tof = open('../pseudo/' + ss + '.tgt.textlabel', 'w')
        ftext = open('../pseudo/' + ss + '.src.text')
        for line, tl in zip(f, ftext):
            lis = [int(a) for b in [x.split(' ') for x in line.strip().split('\t')] for a in b if a]
            res = ['0']*len(tl.strip().split(' '))
            for i in range(0, len(lis), 2):
                res[lis[i]:lis[i+1]] = ['2']*(lis[i+1]-lis[i])
                res[lis[i]] = '1'
            tof.write(' '.join(res)+'\n')
        tof.close()


# check

In [11]:
f1 = open('../pseudo/dev.src.text')
f2 = open('../pseudo/dev.tgt.textlabel')
for l1, l2 in zip(f1, f2):
    if (len(l1.strip().split(' ')) != len(l2.strip().split(' '))):
        print(1)

# data label

In [12]:
sss = ['train', 'dev', 'test']
for ss in sss:
    with open('../pseudo/'+ss+'.src.data') as f:
        tof = open('../pseudo/'+ss+'.tgt.data', 'w')
        for ii, line in enumerate(f):
            line = [x for x in line.strip().split(' ') if x]
            tof.write(' '.join(line)+'\n')
        tof.close()

In [13]:
sss = ['train', 'dev', 'test']
for ss in sss:
    with open('../pseudo/'+ss+'.tgt.data') as f:
        tof = open('../pseudo/'+ss+'.tgt.datalabel', 'w')
        for ii, line in enumerate(f):
            forth, line = line.strip().split('<sep>')
            wdlist = [x for x in line.strip().split(' ') if x]
            forth = [x for x in forth.strip().split(' ') if x]
            lblist = ['0']*len(wdlist)
            for i, s in enumerate(wdlist):
                try:
                    if s[0] != '<' and s[-1] != '>':
                        if lblist[i-1] == '0':
                            lblist[i] = '1'
                        else:
                            lblist[i] = '2'
                except IndexError:
                    print(ii)
                    print(wdlist)
                    print(s)
                    raise
            lblist = ['0']*(len(forth)+1) + lblist
            tof.write(' '.join(lblist)+'\n')
        tof.close()

In [14]:
sss = ['train', 'dev', 'test']
for ss in sss:
    with open('../pseudo/'+ss+'.tgt.datalabel') as f:
        ff = open('../pseudo/'+ss+'.tgt.datapos', 'r')
        tof = open('../pseudo/'+ss+'.tgt.dataptr', 'w')

        for yxj, (label, pos) in enumerate(zip(f, ff)):
            pos = [int(tt) for t in pos.strip().split('\t') for tt in t.strip().split(' ') if tt]
            tolist = []
            nownum = 0
            labellist = label.strip().split(' ')
            labelnum = [-1] * len(labellist)
            for i, n in enumerate(labellist):
                if n == '1':
                    labelnum[i] = nownum                    
                    nownum += 1
            for i in range(0, len(pos), 2):
                try:
                    tolist.append(str(labelnum[pos[i]]))
                except IndexError:
                    print(yxj)
                    print(pos)
                    print(ss)
                    print(labellist)
                    print(labelnum)
                    raise
            tof.write(' '.join(tolist)+'\n')
        tof.close()

# check num

In [11]:
sss = ['train', 'dev', 'test']
for ss in sss:
    with open('../pseudo/'+ss+'.tgt.textlabel') as f:
        f2 = open('../pseudo/'+ss+'.tgt.dataptr')
        for i, (label, ptr) in enumerate(zip(f, f2)):
            if len([x for x in ptr.strip().split(' ') if x]) != label.split(' ').count('1'):
                print(ptr.strip().split(' '), label.split(' ').count('1'))
                print(i, ss)
                break


In [2]:
[1, 2, 1, 3].count(1)

2